In [4]:
import sys
from PyQt5.QtWidgets import * 
from PyQt5.QtGui import * 
import tensorflow as tf
import numpy as np
import keras.utils as image
import cv2
import math
import scipy.ndimage

class MainWindow(QMainWindow):
    
    def __init__(self, parent = None):
       
        super(MainWindow, self).__init__(parent)
        
        menubar = self.menuBar()
        fileMenu = menubar.addMenu('File')
        editMenu = menubar.addMenu('Edit')
        self.setWindowTitle("Merai")
        
        
        self.resize(500, 500)
        
        def center(self):
            frameGm = self.frameGeometry()
            screen = PyQt5.QtWidgets.QApplication.desktop().screenNumber(PyQt5.QtWidgets.QApplication.desktop().cursor().pos())
            centerPoint = PyQt5.QtWidgets.QApplication.desktop().screenGeometry(screen).center()
            frameGm.moveCenter(centerPoint)
            self.move(frameGm.topLeft())

        openAction = QAction('Open Image', self)  
        openAction.triggered.connect(self.openImage) 
        fileMenu.addAction(openAction)

        closeAction = QAction('Exit', self)  
        closeAction.triggered.connect(self.close) 
        fileMenu.addAction(closeAction)
        self.label = QLabel()
        self.setCentralWidget(self.label)
        pixmap = QPixmap(r'C:\Users\Burak\Desktop\YMGK\background\5.jpg')
        self.label.setPixmap(pixmap)
        self.resize(pixmap.size())
        self.adjustSize()
        
        
    def openImage(self):
        imagePath, _ = QFileDialog.getOpenFileName()
        model = tf.keras.models.load_model(r'C:\Users\Burak\Desktop\modelymgk.h5')
        imagePath= self.ozellikCikar(imagePath)
        img = image.load_img(imagePath, target_size = (64, 64))
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis = 0)

        deneme =model.predict(img)
        prediction = deneme
        prediction = prediction[0][0]
        if prediction >=0.5:
            QMessageBox.about(self, "Sonuc","Mermeriniz sağlam")
            print("saglam")
        else:
            QMessageBox.about(self, "Sonuc","Mermeriniz kırık")
            print("kirik")
        
        
        
        
    def ozellikCikar(self,imagepath):
        def orientated_non_max_suppression(mag, ang):
            ang_quant = np.round(ang / (np.pi/4)) % 4
            winE = np.array([[0, 0, 0],[1, 1, 1], [0, 0, 0]])
            winSE = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
            winS = np.array([[0, 1, 0], [0, 1, 0], [0, 1, 0]])
            winSW = np.array([[0, 0, 1], [0, 1, 0], [1, 0, 0]])

            magE = non_max_suppression(mag, winE)
            magSE = non_max_suppression(mag, winSE)
            magS = non_max_suppression(mag, winS)
            magSW = non_max_suppression(mag, winSW)

            mag[ang_quant == 0] = magE[ang_quant == 0]
            mag[ang_quant == 1] = magSE[ang_quant == 1]
            mag[ang_quant == 2] = magS[ang_quant == 2]
            mag[ang_quant == 3] = magSW[ang_quant == 3]
            return mag

        def non_max_suppression(data, win):
            data_max = scipy.ndimage.maximum_filter(data, footprint=win, mode='constant')
            data_max[data != data_max] = 0
            return data_max

        import cv2
        import glob

        gray_image = cv2.imread(imagepath, 0)
        

        
        with_nmsup = True #apply non-maximal suppression
        fudgefactor = 1.3 #with this threshold you can play a little bit
        sigma = 21 #for Gaussian Kernel
        kernel = 2*math.ceil(2*sigma)+1 #Kernel size
        gray_image = gray_image/255.0
        blur = cv2.GaussianBlur(gray_image, (kernel, kernel), sigma)
        gray_image = cv2.subtract(gray_image, blur)

            # compute sobel response
        sobelx = cv2.Sobel(gray_image, cv2.CV_64F, 1, 0, ksize=3)
        sobely = cv2.Sobel(gray_image, cv2.CV_64F, 0, 1, ksize=3)
        mag = np.hypot(sobelx, sobely)            
        ang = np.arctan2(sobely, sobelx)

            # threshold
        threshold = 4 * fudgefactor * np.mean(mag)
        mag[mag < threshold] = 0

            #either get edges directly
        if with_nmsup is False:
            mag = cv2.normalize(mag, 0, 255, cv2.NORM_MINMAX)
            kernel = np.ones((5,5),np.uint8)
            result = cv2.morphologyEx(mag, cv2.MORPH_CLOSE, kernel)
            cv2.imshow('im', result)
                
            cv2.waitKey()

            #or apply a non-maximal suppression
        else:
                # non-maximal suppression
            mag = orientated_non_max_suppression(mag, ang)
                # create mask
            mag[mag > 0] = 255
            mag = mag.astype(np.uint8)

            kernel = np.ones((5,5),np.uint8)
            result = cv2.morphologyEx(mag, cv2.MORPH_CLOSE, kernel)

            
            yol =r'C:\Users\Burak\Desktop\YMGK\denem\de.jpg'
            cv2.imwrite(yol,result)
            return yol
def main():
    app = QApplication(sys.argv)
    win = MainWindow()
    win.show()
    return app.exec_()

if __name__ == '__main__':
    sys.exit(main()) 

SystemExit: 0